In [1]:
import requests
import pandas as pd
from tabulate import tabulate
from IPython.display import HTML

In [59]:
def get_stock_df(ticker):
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=' + ticker + '&apikey=demo'
    r = requests.get(url)
    data = r.json()

    df = pd.DataFrame.from_dict(data, orient='columns')

    df = df.drop("Meta Data", axis = 1)
    df = df.drop(index=["1. Information","2. Symbol","3. Last Refreshed","4. Output Size","5. Time Zone"])

    HTML(tabulate(df, headers='keys', tablefmt='html'))


    df_normalized = pd.json_normalize(df['Time Series (Daily)'])
    df = df.reset_index()
    df_final = df.join(df_normalized)
    df_final.drop('Time Series (Daily)', axis = 1, inplace = True)
    df_final.index = df['index']
    df_final.drop('index', axis = 1, inplace = True)
    return df_final

,1. open,2. high,3. low,4. close,5. volume
index,,,,,
2024-02-20,187.6400,188.7700,183.0600,183.4400,4247181
2024-02-16,186.6300,188.9500,185.9452,187.6400,4842840
2024-02-15,183.6200,186.9800,183.6200,186.8700,4714301
2024-02-14,185.0000,185.0000,182.2600,183.5700,3173391
2024-02-13,184.2800,184.7700,182.3600,183.7000,4290453
...,...,...,...,...,...
2023-10-03,140.8700,141.6400,140.0000,140.3900,3284421
2023-10-02,140.0400,141.4500,139.8600,140.8000,3275461
2023-09-29,142.0000,142.1300,139.6100,140.3000,5703983
